In [83]:
!pip install pandas==1.3.4
!pip install transformers==4.12.5
!pip install datasets==1.15.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [84]:
!pip install ipywidgets
!pip install IProgress

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [85]:
import os
import re
import pickle

import pandas as pd

import numpy as np
import torch

import transformers
from transformers import BertTokenizer

import datasets
from datasets import concatenate_datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import DatasetDict

In [86]:
print('pandas:\t\t', pd.__version__)
print('transformers:\t', transformers.__version__)
print('datasets:\t', datasets.__version__)

pandas:		 1.3.4
transformers:	 4.12.5
datasets:	 1.15.1


## Load Data

In [87]:
dataset_df = pd.read_pickle("/notebooks/KURI-BERT/data/pe_dataset_w_essay_position_pos_tags_pickle")

In [88]:
dataset_df = dataset_df.drop_duplicates(subset=['label_and_comp_idxs', 'text'], keep='first').reset_index(drop=True)

In [89]:
dataset_df = dataset_df.sort_values(['essay_nr', 'argument_bound_1'], ascending=[1, 1]).reset_index(drop=True)

In [90]:
dataset_df

,essay_nr,component_id,label_and_comp_idxs,text,label_x,label_ComponentType,relation_SupportAttack,label_RelationType,label_LinkedNotLinked,split_y,...,is_last_in_para,nr_preceeding_comps_in_para,nr_following_comps_in_para,structural_fts_as_text,structural_fts_as_text_combined,para_ratio,first_or_last,strct_fts_w_position_in_essay,component_pos_tags,strct_fts_essay_position_pos_tags
0,essay001,T1,MajorClaim 503 575,we should attach more importance to cooperatio...,MajorClaim,MajorClaim,[],,Linked,TRAIN,...,1,0,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.25,1,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, VERB, VERB, ADJ, NO...",Topic: Should students be taught to compete or...
1,essay001,T3,Claim 591 714,"through cooperation, children can learn about ...",Claim,Claim,[],Support,Linked,TRAIN,...,0,0,3,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, NOUN, PUNCT, NOUN, V...",Topic: Should students be taught to compete or...
2,essay001,T4,Premise 716 851,What we acquired from team work is not only ho...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,1,2,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: PRON, PRON, VERB, ADP, NO...",Topic: Should students be taught to compete or...
3,essay001,T5,Premise 853 1086,"During the process of cooperation, children ca...",Premise,Premise,[],Support,NotLinked,TRAIN,...,0,2,1,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: ADP, DET, NOUN, ADP, NOUN...",Topic: Should students be taught to compete or...
4,essay001,T6,Premise 1088 1191,All of these skills help them to get on well w...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,3,0,Topic: Should students be taught to compete or...,Topic: Should students be taught to compete or...,0.50,0,Topic: Should students be taught to compete or...,"Part Of Speech tags: DET, ADP, DET, NOUN, VERB...",Topic: Should students be taught to compete or...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5958,essay402,T11,Premise 1275 1339,indirectly they will learn how to socialize ea...,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,4,3,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: ADV, PRON, VERB, VERB, AD...",Topic: Children should studying hard or playin...
5959,essay402,T12,Premise 1341 1388,That will make children getting lots of friends,Premise,Premise,[],Support,NotLinked,TRAIN,...,0,5,2,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: DET, VERB, VERB, NOUN, VE...",Topic: Children should studying hard or playin...
5960,essay402,T13,Premise 1393 1436,they can contribute positively to community,Premise,Premise,[],Support,Linked,TRAIN,...,0,6,1,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: PRON, VERB, VERB, ADV, AD...",Topic: Children should studying hard or playin...
5961,essay402,T14,Premise 1448 1525,playing sport makes children getting healthy a...,Premise,Premise,[],Support,NotLinked,TRAIN,...,1,7,0,Topic: Children should studying hard or playin...,Topic: Children should studying hard or playin...,0.75,0,Topic: Children should studying hard or playin...,"Part Of Speech tags: VERB, NOUN, VERB, NOUN, V...",Topic: Children should studying hard o

In [94]:
dataset_df['text'][456]

'In globalized world, the economies of several countries are in a big network and must influence on each others'

In [9]:
dataset_df['paragraph'][1926] = 'Based on the reasons demonstrated above, I prefer to express my opinion in person because the advantages of this are qualitatively greater than that of the opposite. In other words, I think the best way of communication is by means of face-to-face. It is not to say, of course, that other points of view are completely without merit. Still, I am firmly convinced that the reasons I proposed in favor of my own sentiment are much more tenable and cogent.'

/tmp/ipykernel_1479/867583492.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][1926] = 'Based on the reasons demonstrated above, I prefer to express my opinion in person because the advantages of this are qualitatively greater than that of the opposite. In other words, I think the best way of communication is by means of face-to-face. It is not to say, of course, that other points of view are completely without merit. Still, I am firmly convinced that the reasons I proposed in favor of my own sentiment are much more tenable and cogent.'


In [10]:
dataset_df['paragraph'][4831] = 'Finally, although I prefer to live in a big city, I can not help but assure that this is a matter of personal taste, to be clear, I would love to spend some days in countryside to recharge my soul and relieve myself from daily stress.'

/tmp/ipykernel_1479/3379091247.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][4831] = 'Finally, although I prefer to live in a big city, I can not help but assure that this is a matter of personal taste, to be clear, I would love to spend some days in countryside to recharge my soul and relieve myself from daily stress.'


In [11]:
dataset_df['paragraph'][3908] = 'I do believe that great success requires taking great risks. If you want to gain very high profits in investments, you should use great and very high amounts of money in very risky financial decisions and dealings in which you may lose much amount of money. If you want to achieve a remarkable success in an important exam, you should risk studying all the time and sacrificing your free time and your favorite hobbies. The more you take risks, the greater your successes will be. If you don’t take any risks, you will have an ordinary life with average successes. That’s why it is important to take risks.'

/tmp/ipykernel_1479/2273872724.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][3908] = 'I do believe that great success requires taking great risks. If you want to gain very high profits in investments, you should use great and very high amounts of money in very risky financial decisions and dealings in which you may lose much amount of money. If you want to achieve a remarkable success in an important exam, you should risk studying all the time and sacrificing your free time and your favorite hobbies. The more you take risks, the greater your successes will be. If you don’t take any risks, you will have an ordinary life with average successes. That’s why it is important to take risks.'


In [12]:
dataset_df['paragraph'][4306] = 'In conclusion, experiences that shape my thinking and the way to live my life are teach me to be a better person. Because I could find a better way to solve problems. Besides, experiences could shape me to be more aware about my health. Moreover, these experiences could make me to make plan for my future life. These experiences taught me to change my life to be a better person.'

/tmp/ipykernel_1479/1196760580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_df['paragraph'][4306] = 'In conclusion, experiences that shape my thinking and the way to live my life are teach me to be a better person. Because I could find a better way to solve problems. Besides, experiences could shape me to be more aware about my health. Moreover, these experiences could make me to make plan for my future life. These experiences taught me to change my life to be a better person.'


In [13]:
dataset_df.columns

Index(['essay_nr', 'component_id', 'label_and_comp_idxs', 'text', 'label_x',
       'label_ComponentType', 'relation_SupportAttack', 'label_RelationType',
       'label_LinkedNotLinked', 'split_y', 'essay', 'argument_bound_1',
       'argument_bound_2', 'argument_id', 'sentence', 'paragraph', 'para_nr',
       'total_paras', 'token_count', 'token_count_covering_para',
       'tokens_count_covering_sentence', 'preceeding_tokens_in_sentence_count',
       'succeeding_tokens_in_sentence_count', 'token_ratio',
       'relative_position_in_para_char', 'is_in_intro',
       'relative_position_in_para_token', 'is_in_conclusion',
       'is_first_in_para', 'is_last_in_para', 'nr_preceeding_comps_in_para',
       'nr_following_comps_in_para', 'structural_fts_as_text',
       'structural_fts_as_text_combined', 'para_ratio', 'first_or_last',
       'strct_fts_w_position_in_essay', 'component_pos_tags',
       'strct_fts_essay_position_pos_tags'],
      dtype='object')

In [14]:
dataset_df.structural_fts_as_text_combined[0]

'Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., Structural Features: 1, Yes, Yes, Yes, No'

#### get ams

In [15]:
with open("am_list.txt") as file_am:
    list_of_ams = []
    for line in file_am:
        list_of_ams.append(line.rstrip('\n'))

In [16]:
def am_in_prefix(list_a, list_b):
    
    equal_flag = 0
    
    for i in range(len(list_a)):
        
        sub_list = list_a[i:i+len(list_b)]
        
        if sub_list == list_b:
            
            equal_flag = 1
            break
        
        else:
            
            equal_flag = 0
            
    return equal_flag

In [17]:
def get_am(x, list_of_ams):
    
    am = []
    
    sentence = x.sentence
    component = x.text
    
    prefix = sentence[:sentence.find(component)]
    prefix = re.findall(r'\s|,|[^,\s]+', prefix) 
    prefix = [item for item in prefix if item != " "]
    prefix = [word.casefold() for word in prefix]
    
    list_of_ams = [am.casefold() for am in list_of_ams]
    list_of_ams = list(set(list_of_ams))
  
    for am_phrase in list_of_ams:
        
        am_words_list = re.findall(r'\s|,|[^,\s]+', am_phrase)
        am_words_list = [item for item in am_words_list if item != " "]
        
        if am_in_prefix(prefix, am_words_list) and len(am) == 0:
            
            am.append(am_phrase)
    
    return am         

In [18]:
dataset_df['am'] = dataset_df.apply(lambda x: get_am(x, list_of_ams), axis=1)

In [19]:
dataset_df['am']

0       [from this point of view]
1                  [first of all]
2                              []
3                              []
4                              []
                  ...            
5958                    [because]
5959                           []
5960                         [so]
5961                   [secondly]
5962              [in conclusion]
Name: am, Length: 5963, dtype: object

### do group by for every required column

In [20]:
paras_df = dataset_df.groupby("paragraph", sort=False)

In [21]:
paras_df

In [22]:
df_by_paras_text = paras_df["text"].apply(list)

In [23]:
df_by_paras_text

paragraph
It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.                                                                                                                                                                                                                                                                                           [we should attach more importance to cooperati...
First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all student

In [24]:
df_by_paras_text = df_by_paras_text.reset_index()

In [25]:
df_by_paras_text

,paragraph,text
0,It is always said that competition can effecti...,[we should attach more importance to cooperati...
1,"First of all, through cooperation, children ca...","[through cooperation, children can learn about..."
2,"On the other hand, the significance of competi...",[the significance of competition is that how t...
3,"Consequently, no matter from the view of indiv...",[a more cooperative attitudes towards life is ...
4,The last 50 years have seen an increasing numb...,[they are able to sustain their cultural ident...
...,...,...
1714,"In conclusion, fatherhood should be as present...",[fatherhood should be as present as motherhood...
1715,Some people believe that studying hard is esse...,[both of studying hard and playing sports are ...
1716,"On the other hand, studying hard will give chi...",[studying hard will give children a better fut...
1717,"At the same time, playing sports will give goo...",[playing sports will give good effects on chil...


In [26]:
df_by_paras_labels = paras_df["label_ComponentType"].apply(list).reset_index()

In [27]:
# Nice.

In [28]:
df_by_paras_split = paras_df['split_y'].first()

In [29]:
df_by_paras_split = df_by_paras_split.reset_index()

In [30]:
df_by_paras_split

,paragraph,split_y
0,It is always said that competition can effecti...,TRAIN
1,"First of all, through cooperation, children ca...",TRAIN
2,"On the other hand, the significance of competi...",TRAIN
3,"Consequently, no matter from the view of indiv...",TRAIN
4,The last 50 years have seen an increasing numb...,TRAIN
...,...,...
1714,"In conclusion, fatherhood should be as present...",TRAIN
1715,Some people believe that studying hard is esse...,TRAIN
1716,"On the other hand, studying hard will give chi...",TRAIN
1717,"At the same time, playing sports will give goo...",TRAIN


In [31]:
df_by_paras_ams = paras_df["am"].apply(list)

In [32]:
df_by_paras_ams

paragraph
It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education.                                                                                                                                                                                                                                                                                                                 [[from this point of view]]
First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all student

In [33]:
df_by_paras_ams = df_by_paras_ams.reset_index()

In [34]:
df_by_paras_ams

,paragraph,am
0,It is always said that competition can effecti...,[[from this point of view]]
1,"First of all, through cooperation, children ca...","[[first of all], [], [], []]"
2,"On the other hand, the significance of competi...","[[on the other hand], [hence], [however], [], []]"
3,"Consequently, no matter from the view of indiv...",[[consequently]]
4,The last 50 years have seen an increasing numb...,[[however]]
...,...,...
1714,"In conclusion, fatherhood should be as present...",[[in conclusion]]
1715,Some people believe that studying hard is esse...,[[in my point of view]]
1716,"On the other hand, studying hard will give chi...","[[on the other hand], [], [for instance], [], []]"
1717,"At the same time, playing sports will give goo...","[[], [], [firstly], [], [because], [], [so], [..."


In [35]:
# get essays

In [36]:
df_by_paras_essay = paras_df['essay_nr'].first()

In [37]:
df_by_paras_essay = df_by_paras_essay.reset_index()

In [38]:
df_by_paras_essay

,paragraph,essay_nr
0,It is always said that competition can effecti...,essay001
1,"First of all, through cooperation, children ca...",essay001
2,"On the other hand, the significance of competi...",essay001
3,"Consequently, no matter from the view of indiv...",essay001
4,The last 50 years have seen an increasing numb...,essay002
...,...,...
1714,"In conclusion, fatherhood should be as present...",essay401
1715,Some people believe that studying hard is esse...,essay402
1716,"On the other hand, studying hard will give chi...",essay402
1717,"At the same time, playing sports will give goo...",essay402


In [39]:
# make the first two DFs lists of lists

In [40]:
def make_acs_l_o_l(x):
    
    list_acs = x.text
    
    new_list = [[ac] for ac in list_acs]
    
    return new_list

In [41]:
df_by_paras_text['paragraph_components_list'] = df_by_paras_text.apply(lambda x: make_acs_l_o_l(x), axis=1)

In [42]:
# ok now for the second labels df

In [43]:
def make_labels_l_o_l(x):
    
    list_labels = x.label_ComponentType
    
    new_list = [[label] for label in list_labels]
    
    return new_list

In [44]:
df_by_paras_labels['paragraph_labels_list'] = df_by_paras_labels.apply(lambda x: make_labels_l_o_l(x), axis=1)

In [45]:
# ok seems correct.

In [46]:
# new df for strct fts

In [47]:
# df_by_paras_strct_fts = paras_df["structural_fts_as_text"].apply(list)
df_by_paras_strct_fts = paras_df["structural_fts_as_text_combined"].apply(list)

In [48]:
df_by_paras_strct_fts = df_by_paras_strct_fts.reset_index(drop=True)

In [49]:
# now combine the dataframes

In [50]:
df = pd.concat([df_by_paras_text, df_by_paras_labels, df_by_paras_ams, df_by_paras_split, df_by_paras_essay, df_by_paras_strct_fts], axis=1)

In [51]:
df

,paragraph,text,paragraph_components_list,paragraph,label_ComponentType,paragraph_labels_list,paragraph,am,paragraph,split_y,paragraph,essay_nr,structural_fts_as_text_combined
0,It is always said that competition can effecti...,[we should attach more importance to cooperati...,[[we should attach more importance to cooperat...,It is always said that competition can effecti...,[MajorClaim],[[MajorClaim]],It is always said that competition can effecti...,[[from this point of view]],It is always said that competition can effecti...,TRAIN,It is always said that competition can effecti...,essay001,[Topic: Should students be taught to compete o...
1,"First of all, through cooperation, children ca...","[through cooperation, children can learn about...","[[through cooperation, children can learn abou...","First of all, through cooperation, children ca...","[Claim, Premise, Premise, Premise]","[[Claim], [Premise], [Premise], [Premise]]","First of all, through cooperation, children ca...","[[first of all], [], [], []]","First of all, through cooperation, children ca...",TRAIN,"First of all, through cooperation, children ca...",essay001,[Topic: Should students be taught to compete o...
2,"On the other hand, the significance of competi...",[the significance of competition is that how t...,[[the significance of competition is that how ...,"On the other hand, the significance of competi...","[Premise, Claim, Premise, Premise, Claim]","[[Premise], [Claim], [Premise], [Premise], [Cl...","On the other hand, the significance of competi...","[[on the other hand], [hence], [however], [], []]","On the other hand, the significance of competi...",TRAIN,"On the other hand, the significance of competi...",essay001,[Topic: Should students be taught to compete o...
3,"Consequently, no matter from the view of indiv...",[a more cooperative attitudes towards life is ...,[[a more cooperative attitudes towards life is...,"Consequently, no matter from the view of indiv...",[MajorClaim],[[MajorClaim]],"Consequently, no matter from the view of indiv...",[[consequently]],"Consequently, no matter from the view of indiv...",TRAIN,"Consequently, no matter from the view of indiv...",essay001,[Topic: Should students be taught to compete o...
4,The last 50 years have seen an increasing numb...,[they are able to sustain their cultural ident...,[[they are able to sustain their cultural iden...,The last 50 years have seen an increasing numb...,[MajorClaim],[[MajorClaim]],The last 50 years have seen an increasing numb...,[[however]],The last 50 years have seen an increasing numb...,TRAIN,The last 50 years have seen an increasing numb...,essay002,[Topic: More people are migrating to other cou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[fatherhood should be as present as motherhood...,[[fatherhood should be as present as motherhoo...,"In conclusion, fatherhood should be as present...",[MajorClaim],[[MajorClaim]],"In conclusion, fatherhood should be as present...",[[in conclusion]],"In conclusion, fatherhood should be as present...",TRAIN,"In conclusion, fatherhood should be as present...",essay401,[Topic: Fatherhood should be as present as mot...
1715,Some people believe that studying hard is esse...,[both of studying hard and playing sports are ...,[[both of studying hard and playing sports are...,Some people believe that studying hard is esse...,[MajorClaim],[[MajorClaim]],Some people believe that studying hard is esse...,[[in my point of view]],Some people believe that studying hard is esse...,TRAIN,Some people believe that studying hard is esse...,essay402,[Topic: Children should studying hard or playi...
1716,"On the other hand, studying hard will give chi...",[studying hard will give children a better fut...,[[studying hard will give children a better fu...,"On the other hand, studying hard will give chi...","[Claim, Premise, Premise, Premise, Premise]","[[Claim], [Premise], [Premise], [Premise], 

In [52]:
# df['structural_fts_as_text'][6]

In [53]:
df = df.loc[:,~df.columns.duplicated()].copy()

In [54]:
df

,paragraph,text,paragraph_components_list,label_ComponentType,paragraph_labels_list,am,split_y,essay_nr,structural_fts_as_text_combined
0,It is always said that competition can effecti...,[we should attach more importance to cooperati...,[[we should attach more importance to cooperat...,[MajorClaim],[[MajorClaim]],[[from this point of view]],TRAIN,essay001,[Topic: Should students be taught to compete o...
1,"First of all, through cooperation, children ca...","[through cooperation, children can learn about...","[[through cooperation, children can learn abou...","[Claim, Premise, Premise, Premise]","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...
2,"On the other hand, the significance of competi...",[the significance of competition is that how t...,[[the significance of competition is that how ...,"[Premise, Claim, Premise, Premise, Claim]","[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...
3,"Consequently, no matter from the view of indiv...",[a more cooperative attitudes towards life is ...,[[a more cooperative attitudes towards life is...,[MajorClaim],[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...
4,The last 50 years have seen an increasing numb...,[they are able to sustain their cultural ident...,[[they are able to sustain their cultural iden...,[MajorClaim],[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...
...,...,...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[fatherhood should be as present as motherhood...,[[fatherhood should be as present as motherhoo...,[MajorClaim],[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...
1715,Some people believe that studying hard is esse...,[both of studying hard and playing sports are ...,[[both of studying hard and playing sports are...,[MajorClaim],[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...
1716,"On the other hand, studying hard will give chi...",[studying hard will give children a better fut...,[[studying hard will give children a better fu...,"[Claim, Premise, Premise, Premise, Premise]","[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...
1717,"At the same time, playing sports will give goo...",[playing sports will give good effects on chil...,[[playing sports will give good effects on chi...,"[Claim, Premise, Premise, Premise, Premise, Pr...","[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...


In [55]:
df = df.drop(columns=['text', 'label_ComponentType'])

In [56]:
df = df.rename(columns={"am": "paragraph_markers_list"})

In [57]:
df = df.rename(columns={"split_y": "split"})

In [58]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,split,essay_nr,structural_fts_as_text_combined
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[from this point of view]],TRAIN,essay001,[Topic: Should students be taught to compete o...
1,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...
2,"On the other hand, the significance of competi...",[[the significance of competition is that how ...,"[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...
3,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...
...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[[fatherhood should be as present as motherhoo...,[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...
1716,"On the other hand, studying hard will give chi...",[[studying hard will give children a better fu...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...
1717,"At the same time, playing sports will give goo...",[[playing sports will give good effects on chi...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...


In [59]:
def is_equal(x):
    
    # flag = 0
    
    list_ac = x.paragraph_components_list
    list_am = x.paragraph_markers_list
    
    return len(list_ac) == len(list_am)

In [60]:
df['ac=am'] = df.apply(lambda x: is_equal(x), axis=1)

In [61]:
df['ac=am'].value_counts()

True    1719
Name: ac=am, dtype: int64

In [62]:
def is_equal_label(x):
    
    flag = 0
    
    list_ac = x.paragraph_components_list
    list_am = x.paragraph_labels_list
    
    if len(list_ac) == len(list_am):
        
        flag = 1
        
    else:
        
        flag = 0
        
    return flag

In [63]:
df['ac=lab'] = df.apply(lambda x: is_equal_label(x), axis=1)

In [64]:
df['ac=lab'].value_counts()

1    1719
Name: ac=lab, dtype: int64

In [65]:
# ok both correct.

In [66]:
df = df.drop(columns=df.columns[-1])

In [67]:
df = df.drop(columns=df.columns[-1])

In [68]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,split,essay_nr,structural_fts_as_text_combined
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[from this point of view]],TRAIN,essay001,[Topic: Should students be taught to compete o...
1,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...
2,"On the other hand, the significance of competi...",[[the significance of competition is that how ...,"[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...
3,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...
...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[[fatherhood should be as present as motherhoo...,[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...
1716,"On the other hand, studying hard will give chi...",[[studying hard will give children a better fu...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...
1717,"At the same time, playing sports will give goo...",[[playing sports will give good effects on chi...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...


In [69]:
def get_numeric_labels(x):
    
    numeric_labels = []
    
    string_labels = x.paragraph_labels_list
    
    for label in string_labels:
        
        if label == ['MajorClaim']:
            
            numeric_labels.append(0)
            
        elif label == ['Claim']:
            
            numeric_labels.append(1)
            
        elif label == ['Premise']:
            
            numeric_labels.append(2)
            
    return numeric_labels

In [70]:
df['paragraph_labels_numeric'] = df.apply(lambda x: get_numeric_labels(x), axis=1)

In [71]:
# correct the five problematic things in the DF.

In [72]:
df['paragraph_components_list'][409] = [['when children take jobs, they tend to be more responsible'],
 ['whether they can earn money or not will depend on their effectiveness and attitudes in working'],
 ['To keep their jobs, children will have to try their best to finish their duties'],
 ['if a child works as a shop assistant, he will have to keep an eye on good and products, calculate prices correctly and take responsibilities for any lost']]

In [73]:
df['paragraph_components_list'][473] = [['children need human interactions'],
 ['No matter how advanced technology will be, it is clear that a computer will never be able to communicate with its pupils'],
 ["As a recent interview with Josh Trumm – the founder of the 'Teachers Global Group'- reveals, human interactions will never be imitated by computers: 'This verbal and non-verbal communication between teachers and children is the first step towards a proper education"],
 ["Without someone who can explain and understand their problems, pupils cannot actually learn'"],
 ['only a teacher can feel and connect with pupils, be a friend and a model']]

In [74]:
df['paragraph_components_list'][771] = [['the individual should finance his or her education'],
 ['it would cause too much cost from taxpayers and the government']]

In [75]:
df['paragraph_components_list'][1061] = [['as parents are involved in the same environment'],
 ['they could thus support much more to their children, such as domain knowledge and industry-related information'],
 ['my father is a software programmer, and he indeed teaches me a lot in programming and some computer knowledge'],
 ['Whenever I met a difficulty, I would reach him for consulting'],
 ['he has many experiences'],
 ['I could save enormous time in finding solution'],
 ['This truly makes me more competitive than my colleagues'],
 ['If I worked in the different field from my father, I would have faced much difficulty than I do now']]

In [76]:
df['paragraph_components_list'][1470] = [['With rapid development of the internet, people are able to enjoy quick electronic communication via internet'],
 ['as lots of chatting apps available online such as "Weichat", people tend to send instant messages free of charge by using their phones rather than face to face communication'],
 ['mobile phones have shortened the distance of communication'],
 ['mobile phones have shortened the distance of communication']]

In [77]:
df['paragraph_components_list'][1470] = [['With rapid development of the internet, people are able to enjoy quick electronic communication via internet'],
  ['as lots of chatting apps available online such as "Weichat", people tend to send instant messages free of charge by using their phones rather than face to face communication'],
  ['mobile phones have shortened the distance of communication']]

In [78]:
df['paragraph_markers_list'][1470] = [[], ['for instance'], ['therefore,']]

In [79]:
# paragraph with the strct fts as text inserted after the components
# ac spans, am spans, fts spans

In [80]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,split,essay_nr,structural_fts_as_text_combined,paragraph_labels_numeric
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[from this point of view]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0]
1,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[1, 2, 2, 2]"
2,"On the other hand, the significance of competi...",[[the significance of competition is that how ...,"[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[2, 1, 2, 2, 1]"
3,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0]
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...,[0]
...,...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[[fatherhood should be as present as motherhoo...,[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...,[0]
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...,[0]
1716,"On the other hand, studying hard will give chi...",[[studying hard will give children a better fu...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2]"
1717,"At the same time, playing sports will give goo...",[[playing sports will give good effects on chi...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2, 2, 2, 2]"


In [81]:
df.to_pickle('df_pe_for_kuri_model')

## get spans

In [81]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [82]:
def get_ac_spans(x):
    
    paragraph = x.paragraph
    paragraph_tokenized = tokenizer.tokenize(paragraph)
    
    ac_list = x.paragraph_components_list
    
    ac_spans = []        
                
    for ac in ac_list:            

        ac_tokenized = tokenizer.tokenize(ac[0])
        
        for i in range(len(paragraph_tokenized)):
            
            span = (i, i+len(ac_tokenized)-1)
            
            if paragraph_tokenized[span[0]:span[1]+1] == ac_tokenized and span not in ac_spans:
                
                ac_spans.append(span)    
                 
    return ac_spans

In [83]:
df['ac_spans'] = df.apply(lambda x: get_ac_spans(x), axis=1)

In [84]:
ac_spans_list = df['ac_spans'].tolist()

In [85]:
def get_inter_span_intervals(ac_spans):
    
    if ac_spans[0][0] == 0:
        
        ac_inter_spans = [(0, 0)]
        
    else:
        
        ac_inter_spans = [(0, ac_spans[0][0]-1)]
      
    ac_inter_spans += [(ac_spans[i][1] + 1, ac_spans[i+1][0]-1) for i in range(len(ac_spans)-1)]
    
    return ac_inter_spans

In [86]:
def get_am_spans(x):
    
    
    # df_idx = df.index[df['paragraph'] == x.paragraph]
    # print(df_idx[0])
    paragraph = x.paragraph
    paragraph_tokenized = tokenizer.tokenize(paragraph)
    
    am_list = x.paragraph_markers_list
    
    ac_spans = x.ac_spans
    ac_inter_spans = get_inter_span_intervals(ac_spans)
    # print(ac_inter_spans)
    am_spans = [(-1, -1)] * len(am_list)
                
    for idx, am in enumerate(am_list):
        inter_span =  ac_inter_spans[idx] 
        
        if am:
            
            am_tokenized = tokenizer.tokenize(am[0])

            for i in range(inter_span[0], inter_span[1]):

                ac_prefix = paragraph_tokenized[inter_span[0]:inter_span[1]]

                span = (i, i+len(am_tokenized)-1)

                if paragraph_tokenized[span[0]:span[1]+1] == am_tokenized:
                
                    am_spans[idx] = span
    
    return am_spans

In [87]:
df['am_spans'] = df.apply(lambda x: get_am_spans(x), axis=1)

In [88]:
# correct! 79 to 387 is the correction work we need.
# SANITY CHECK BELOW

In [89]:
df['am_spans'], df['ac_spans']

(0                                              [(83, 86)]
 1                  [(0, 2), (-1, -1), (-1, -1), (-1, -1)]
 2        [(0, 3), (21, 21), (34, 34), (-1, -1), (-1, -1)]
 3                                                [(0, 0)]
 4                                              [(54, 54)]
                               ...                        
 1714                                             [(0, 1)]
 1715                                           [(24, 28)]
 1716     [(0, 3), (-1, -1), (42, 43), (-1, -1), (-1, -1)]
 1717    [(-1, -1), (-1, -1), (41, 41), (-1, -1), (-1, ...
 1718                                             [(0, 1)]
 Name: am_spans, Length: 1719, dtype: object,
 0                                              [(87, 96)]
 1                [(4, 23), (25, 51), (53, 94), (96, 116)]
 2       [(5, 19), (27, 32), (36, 56), (70, 131), (148,...
 3                                              [(24, 37)]
 4                                              [(59, 75)]
          

In [90]:
def sanity_check(x):
    
    return len(x.ac_spans) == len(x.am_spans) 

In [91]:
df['sanity'] = df.apply(lambda x: sanity_check(x), axis=1)

In [92]:
df['sanity'].value_counts()

True    1719
Name: sanity, dtype: int64

In [93]:
# nice! correct. now all am_spans and ac_spans are equal in length.

In [94]:
df = df.drop(columns=df.columns[-1])

In [95]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,split,essay_nr,structural_fts_as_text_combined,paragraph_labels_numeric,ac_spans,am_spans
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[i firmly believe that]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(87, 96)]","[(83, 86)]"
1,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[1, 2, 2, 2]","[(4, 23), (25, 51), (53, 94), (96, 116)]","[(0, 2), (-1, -1), (-1, -1), (-1, -1)]"
2,"On the other hand, the significance of competi...",[[the significance of competition is that how ...,"[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[2, 1, 2, 2, 1]","[(5, 19), (27, 32), (36, 56), (70, 131), (148,...","[(0, 3), (21, 21), (34, 34), (-1, -1), (-1, -1)]"
3,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(24, 37)]","[(0, 0)]"
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...,[0],"[(59, 75)]","[(54, 54)]"
...,...,...,...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[[fatherhood should be as present as motherhoo...,[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...,[0],"[(3, 33)]","[(0, 1)]"
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...,[0],"[(30, 42)]","[(24, 28)]"
1716,"On the other hand, studying hard will give chi...",[[studying hard will give children a better fu...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2]","[(5, 12), (14, 27), (45, 75), (77, 91), (96, 1...","[(0, 3), (-1, -1), (42, 43), (-1, -1), (-1, -1)]"
1717,"At the same time, playing sports will give goo...",[[playing sports will give good effects on chi...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2, 2, 2, 2]","[(5, 12), (32, 39), (43, 64), (66, 71), (74, 8...","[(-1, -1), (-1, -1), (41, 41), (-1, -1), (-1, ..."


In [96]:
df = df.rename(columns={'am_spans' : 'paragraph_am_spans', 'ac_spans' : 'paragraph_ac_spans', 'paragraph_labels_numeric' : 'paragraph_labels'})

In [97]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,split,essay_nr,structural_fts_as_text_combined,paragraph_labels,paragraph_ac_spans,paragraph_am_spans
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[i firmly believe that]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(87, 96)]","[(83, 86)]"
1,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[1, 2, 2, 2]","[(4, 23), (25, 51), (53, 94), (96, 116)]","[(0, 2), (-1, -1), (-1, -1), (-1, -1)]"
2,"On the other hand, the significance of competi...",[[the significance of competition is that how ...,"[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[2, 1, 2, 2, 1]","[(5, 19), (27, 32), (36, 56), (70, 131), (148,...","[(0, 3), (21, 21), (34, 34), (-1, -1), (-1, -1)]"
3,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(24, 37)]","[(0, 0)]"
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...,[0],"[(59, 75)]","[(54, 54)]"
...,...,...,...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[[fatherhood should be as present as motherhoo...,[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...,[0],"[(3, 33)]","[(0, 1)]"
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...,[0],"[(30, 42)]","[(24, 28)]"
1716,"On the other hand, studying hard will give chi...",[[studying hard will give children a better fu...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2]","[(5, 12), (14, 27), (45, 75), (77, 91), (96, 1...","[(0, 3), (-1, -1), (42, 43), (-1, -1), (-1, -1)]"
1717,"At the same time, playing sports will give goo...",[[playing sports will give good effects on chi...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2, 2, 2, 2]","[(5, 12), (32, 39), (43, 64), (66, 71), (74, 8...","[(-1, -1), (-1, -1), (41, 41), (-1, -1), (-1, ..."


In [98]:
# sanity check: check if nr of paragraph strcts fts are the same as nr of paragraph ac spans for all paragraphs

In [99]:
def sanity_check(x):
    
    return len(x.structural_fts_as_text) == len(x.paragraph_ac_spans) 

In [100]:
# df['sanity'] = df.apply(lambda x: sanity_check(x), axis=1)

In [101]:
#df['sanity'].value_counts()

In [102]:
# def sanity_check(x):
    
#     return len(x.paragraph_components_list) == len(x.structural_fts_as_text) 

In [103]:
#df['sanity_2'] = df.apply(lambda x: sanity_check(x), axis=1)

In [104]:
#df['sanity_2'].value_counts()

In [105]:
# ok nice. both sanity checks done.
# now delete these extra columns

In [106]:
# df = df.drop(columns=df.columns[-1])
# df = df.drop(columns=df.columns[-1])

In [107]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,split,essay_nr,structural_fts_as_text_combined,paragraph_labels,paragraph_ac_spans,paragraph_am_spans
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[i firmly believe that]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(87, 96)]","[(83, 86)]"
1,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[1, 2, 2, 2]","[(4, 23), (25, 51), (53, 94), (96, 116)]","[(0, 2), (-1, -1), (-1, -1), (-1, -1)]"
2,"On the other hand, the significance of competi...",[[the significance of competition is that how ...,"[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[2, 1, 2, 2, 1]","[(5, 19), (27, 32), (36, 56), (70, 131), (148,...","[(0, 3), (21, 21), (34, 34), (-1, -1), (-1, -1)]"
3,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(24, 37)]","[(0, 0)]"
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...,[0],"[(59, 75)]","[(54, 54)]"
...,...,...,...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[[fatherhood should be as present as motherhoo...,[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...,[0],"[(3, 33)]","[(0, 1)]"
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...,[0],"[(30, 42)]","[(24, 28)]"
1716,"On the other hand, studying hard will give chi...",[[studying hard will give children a better fu...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2]","[(5, 12), (14, 27), (45, 75), (77, 91), (96, 1...","[(0, 3), (-1, -1), (42, 43), (-1, -1), (-1, -1)]"
1717,"At the same time, playing sports will give goo...",[[playing sports will give good effects on chi...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2, 2, 2, 2]","[(5, 12), (32, 39), (43, 64), (66, 71), (74, 8...","[(-1, -1), (-1, -1), (41, 41), (-1, -1), (-1, ..."


In [108]:
df = df.rename(columns={'structural_fts_as_text_combined' : 'paragraph_fts_as_txt'})

In [109]:
# now create the two new columns needed.
# first strcts as fts span.
# second get the paragraph concated with strct fts.

In [110]:
df

,paragraph,paragraph_components_list,paragraph_labels_list,paragraph_markers_list,split,essay_nr,paragraph_fts_as_txt,paragraph_labels,paragraph_ac_spans,paragraph_am_spans
0,It is always said that competition can effecti...,[[we should attach more importance to cooperat...,[[MajorClaim]],[[i firmly believe that]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(87, 96)]","[(83, 86)]"
1,"First of all, through cooperation, children ca...","[[through cooperation, children can learn abou...","[[Claim], [Premise], [Premise], [Premise]]","[[first of all], [], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[1, 2, 2, 2]","[(4, 23), (25, 51), (53, 94), (96, 116)]","[(0, 2), (-1, -1), (-1, -1), (-1, -1)]"
2,"On the other hand, the significance of competi...",[[the significance of competition is that how ...,"[[Premise], [Claim], [Premise], [Premise], [Cl...","[[on the other hand], [hence], [however], [], []]",TRAIN,essay001,[Topic: Should students be taught to compete o...,"[2, 1, 2, 2, 1]","[(5, 19), (27, 32), (36, 56), (70, 131), (148,...","[(0, 3), (21, 21), (34, 34), (-1, -1), (-1, -1)]"
3,"Consequently, no matter from the view of indiv...",[[a more cooperative attitudes towards life is...,[[MajorClaim]],[[consequently]],TRAIN,essay001,[Topic: Should students be taught to compete o...,[0],"[(24, 37)]","[(0, 0)]"
4,The last 50 years have seen an increasing numb...,[[they are able to sustain their cultural iden...,[[MajorClaim]],[[however]],TRAIN,essay002,[Topic: More people are migrating to other cou...,[0],"[(59, 75)]","[(54, 54)]"
...,...,...,...,...,...,...,...,...,...,...
1714,"In conclusion, fatherhood should be as present...",[[fatherhood should be as present as motherhoo...,[[MajorClaim]],[[in conclusion]],TRAIN,essay401,[Topic: Fatherhood should be as present as mot...,[0],"[(3, 33)]","[(0, 1)]"
1715,Some people believe that studying hard is esse...,[[both of studying hard and playing sports are...,[[MajorClaim]],[[in my point of view]],TRAIN,essay402,[Topic: Children should studying hard or playi...,[0],"[(30, 42)]","[(24, 28)]"
1716,"On the other hand, studying hard will give chi...",[[studying hard will give children a better fu...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[on the other hand], [], [for instance], [], []]",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2]","[(5, 12), (14, 27), (45, 75), (77, 91), (96, 1...","[(0, 3), (-1, -1), (42, 43), (-1, -1), (-1, -1)]"
1717,"At the same time, playing sports will give goo...",[[playing sports will give good effects on chi...,"[[Claim], [Premise], [Premise], [Premise], [Pr...","[[], [], [firstly], [], [because], [], [so], [...",TRAIN,essay402,[Topic: Children should studying hard or playi...,"[1, 2, 2, 2, 2, 2, 2, 2]","[(5, 12), (32, 39), (43, 64), (66, 71), (74, 8...","[(-1, -1), (-1, -1), (41, 41), (-1, -1), (-1, ..."


In [111]:
df.columns

Index(['paragraph', 'paragraph_components_list', 'paragraph_labels_list',
       'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_fts_as_txt',
       'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans'],
      dtype='object')

In [112]:
df = df.rename(columns={"paragraph_fts_as_txt": "paragraph_fts_as_txt_list"})

In [113]:
df.columns

Index(['paragraph', 'paragraph_components_list', 'paragraph_labels_list',
       'paragraph_markers_list', 'split', 'essay_nr',
       'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans',
       'paragraph_am_spans'],
      dtype='object')

In [114]:
df['paragraph_components_list'][769] = [['it is a personal choice to go to a college'],
 ['Not everyone wants to go to a college'],
 ['People who want to make more money, and live a better life than other people decide to take a college education'],
 ['those people should make the financial investment in their own choice; not the government']]

# hackey correction to equalize acs and spans.

In [115]:
df['paragraph_markers_list'][769] = [[], [], [], ['therefore,']]

# hack to correct unequal am lists

In [116]:
# df['paragraph_fts_as_txt_list'][769] = ['Topic: Should the Government Provide Free College?, Sentence: Second, it is a personal choice to go to a college, Para Number: 3, First in Para: Yes, Last in Para: No, Is in Introduction: No, Is in Conclusion: No',
#  'Topic: Should the Government Provide Free College?, Sentence: Not everyone wants to go to a college, Para Number: 3, First in Para: No, Last in Para: No, Is in Introduction: No, Is in Conclusion: No',
#  'Topic: Should the Government Provide Free College?, Sentence: People who want to make more money, and live a better life than other people decide to take a college education, Para Number: 3, First in Para: No, Last in Para: No, Is in Introduction: No, Is in Conclusion: No',
#  'Topic: Should the Government Provide Free College?, Sentence: Therefore, those people should make the financial investment in their own choice; not the government., Para Number: 3, First in Para: No, Last in Para: No, Is in Introduction: No, Is in Conclusion: No']

In [117]:
# df['paragraph_fts_as_txt_list'][786] = ['Topic: Farmland, housing and industry or saving land for animals?, Sentence: Nowadays, there is a prevailing opinion that human needs for farmland, housing and industry are more important than saving land for endangered animals, Para Number: 1, First in Para: Yes, Last in Para: No, Is in Introduction: Yes, Is in Conclusion: No',
#  'Topic: Farmland, housing and industry or saving land for animals?, Sentence: Nowadays, there is a prevailing opinion that human needs for farmland, housing and industry are more important than saving land for endangered animals, Para Number: 1, First in Para: No, Last in Para: Yes, Is in Introduction: Yes, Is in Conclusion: No']

# # correction for exactly similar component

In [118]:
df['paragraph_fts_as_txt_list'][1717]

['Topic: Children should studying hard or playing sport? Both can improve their future, Sentence: At the same time, playing sports will give good effects on children, Structural Features: 3, Yes, No, No, No',
 'Topic: Children should studying hard or playing sport? Both can improve their future, Sentence: Also, there are many benefit when children doing sport, Structural Features: 3, No, No, No, No',
 'Topic: Children should studying hard or playing sport? Both can improve their future, Sentence: Firstly, children who always playing sport like football, they will be active as a person and learn to become team-player, Structural Features: 3, No, No, No, No',
 'Topic: Children should studying hard or playing sport? Both can improve their future, Sentence: It will be good for children, because indirectly they will learn how to socialize each other as a team, Structural Features: 3, No, No, No, No',
 'Topic: Children should studying hard or playing sport? Both can improve their future, Sen

In [119]:
def get_paras_w_fts(x):
    
    paragraph = x.paragraph
    para_fts_as_txt = x.paragraph_fts_as_txt_list
    para_acs = x.paragraph_components_list
    para_ams = x.paragraph_markers_list
    # para_fts_as_txt_spans = x.paragraph_fts_as_txt_spans
    
    # print("len of list:", len(para_fts_as_txt))
    
    for am, ac, ac_fts_as_txt in zip(para_ams, para_acs, para_fts_as_txt):
        

        print('printy: ', ac_fts_as_txt)
        
        para_nr_idx = ac_fts_as_txt.index("Structural Features:")
        ac_fts_as_txt = ac_fts_as_txt[para_nr_idx:]
        
        fts_as_txt_new = ' [SEP] ' + ac_fts_as_txt + ' [SEP]'
        
        paragraph = paragraph.replace(ac[0], ac[0] + fts_as_txt_new, 1)
        # paragraph = new_para

        
    return paragraph    

In [120]:
# problematic. because of the last line, there is repetition.
# oh no. maybe it's correct.

In [121]:
df['paragraph_w_fts_as_txt'] = df.apply(lambda x: get_paras_w_fts(x), axis=1)

printy:  Topic: Should students be taught to compete or to cooperate?, Sentence: From this point of view, I firmly believe that we should attach more importance to cooperation during primary education., Structural Features: 1, Yes, Yes, Yes, No
printy:  Topic: Should students be taught to compete or to cooperate?, Sentence: First of all, through cooperation, children can learn about interpersonal skills which are significant in the future life of all students, Structural Features: 2, Yes, No, No, No
printy:  Topic: Should students be taught to compete or to cooperate?, Sentence: What we acquired from team work is not only how to achieve the same goal with others but more importantly, how to get along with others, Structural Features: 2, No, No, No, No
printy:  Topic: Should students be taught to compete or to cooperate?, Sentence: During the process of cooperation, children can learn about how to listen to opinions of others, how to communicate with others, how to think comprehensively

In [122]:
df['paragraph_w_fts_as_txt'][0]

"It is always said that competition can effectively promote the development of economy. In order to survive in the competition, companies continue to improve their products and service, and as a result, the whole society prospers. However, when we discuss the issue of competition or cooperation, what we are concerned about is not the whole society, but the development of an individual's whole life. From this point of view, I firmly believe that we should attach more importance to cooperation during primary education [SEP] Structural Features: 1, Yes, Yes, Yes, No [SEP]."

In [123]:
df['paragraph_w_fts_as_txt'][786] = 'Nowadays, there is a prevailing opinion that human needs for farmland, housing and industry are more important [SEP] Structural Features: 1, Yes, Yes, Yes, No [SEP] than saving land for endangered animals. People who disagree with the point dispute that the decreasing of land for endangered animals will bring damage to ecological balance. As far as I am concerned, I agree with the opinion human needs for farmland, housing and industry are more important [SEP] 1, No, Yes, Yes, No [SEP]. The reasons are based on the following aspects.'

In [124]:
# now. 
# 1. find am_spans
# 2. find ac_spans
# 3. find fts_spans

In [125]:
def get_ac_spans_new(x):
    
    paragraph = x.paragraph_w_fts_as_txt
    paragraph_tokenized = tokenizer.tokenize(paragraph)
    
    ac_list = x.paragraph_components_list
    
    ac_spans = []        
                
    for ac in ac_list:            

        ac_tokenized = tokenizer.tokenize(ac[0])
        
        for i in range(len(paragraph_tokenized)):
            
            span = (i, i+len(ac_tokenized)-1)
            
            if paragraph_tokenized[span[0]:span[1]+1] == ac_tokenized and span not in ac_spans:
                
                ac_spans.append(span)    
                 
    return ac_spans

In [126]:
df['ac_spans_new'] = df.apply(lambda x: get_ac_spans_new(x), axis=1)

In [127]:
def sanity_check(x):
    
    return len(x.ac_spans_new) == len(x.paragraph_components_list) 

In [128]:
df['sanity_new'] = df.apply(lambda x: sanity_check(x), axis=1)

In [129]:
df['sanity_new'].value_counts()

True    1719
Name: sanity_new, dtype: int64

In [130]:
def get_inter_span_intervals_new(ac_spans):
    
    if ac_spans[0][0] == 0:
        
        ac_inter_spans = [(0, 0)]
        
    else:
        
        ac_inter_spans = [(0, ac_spans[0][0]-1)]
      
    ac_inter_spans += [(ac_spans[i][1] + 1, ac_spans[i+1][0]-1) for i in range(len(ac_spans)-1)]
    
    return ac_inter_spans

In [131]:
def get_am_spans_new(x):
    
    
    # df_idx = df.index[df['paragraph_w_fts_as_txt'] == x.paragraph_w_fts_as_txt]
    # print(df_idx[0])
    paragraph = x.paragraph_w_fts_as_txt
    paragraph_tokenized = tokenizer.tokenize(paragraph)
    
    am_list = x.paragraph_markers_list
    
    ac_spans = x.ac_spans_new
    ac_inter_spans = get_inter_span_intervals_new(ac_spans)
    # print(ac_inter_spans)
    am_spans = [(-1, -1)] * len(am_list)
                
    for idx, am in enumerate(am_list):
        
        inter_span =  ac_inter_spans[idx] 
        
        if am:
            
            am_tokenized = tokenizer.tokenize(am[0])

            for i in range(inter_span[0], inter_span[1]):

                ac_prefix = paragraph_tokenized[inter_span[0]:inter_span[1]]

                span = (i, i+len(am_tokenized)-1)

                if paragraph_tokenized[span[0]:span[1]+1] == am_tokenized:
                
                    am_spans[idx] = span
    
    return am_spans

In [132]:
df['am_spans_new'] = df.apply(lambda x: get_am_spans_new(x), axis=1)

In [133]:
df['am_spans_new']

0                                              [(83, 86)]
1                  [(0, 2), (-1, -1), (-1, -1), (-1, -1)]
2        [(0, 3), (35, 35), (62, 62), (-1, -1), (-1, -1)]
3                                                [(0, 0)]
4                                              [(54, 54)]
                              ...                        
1714                                             [(0, 1)]
1715                                           [(24, 28)]
1716     [(0, 3), (-1, -1), (70, 71), (-1, -1), (-1, -1)]
1717    [(-1, -1), (-1, -1), (69, 69), (-1, -1), (-1, ...
1718                                             [(0, 1)]
Name: am_spans_new, Length: 1719, dtype: object

In [134]:
df.columns

Index(['paragraph', 'paragraph_components_list', 'paragraph_labels_list',
       'paragraph_markers_list', 'split', 'essay_nr',
       'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans',
       'paragraph_am_spans', 'paragraph_w_fts_as_txt', 'ac_spans_new',
       'sanity_new', 'am_spans_new'],
      dtype='object')

In [135]:
df['paragraph_fts_as_txt_list']

0       [Topic: Should students be taught to compete o...
1       [Topic: Should students be taught to compete o...
2       [Topic: Should students be taught to compete o...
3       [Topic: Should students be taught to compete o...
4       [Topic: More people are migrating to other cou...
                              ...                        
1714    [Topic: Fatherhood should be as present as mot...
1715    [Topic: Children should studying hard or playi...
1716    [Topic: Children should studying hard or playi...
1717    [Topic: Children should studying hard or playi...
1718    [Topic: Children should studying hard or playi...
Name: paragraph_fts_as_txt_list, Length: 1719, dtype: object

In [136]:
def get_fts_spans_new(x):
    
    paragraph = x.paragraph_w_fts_as_txt
    paragraph_tokenized = tokenizer.tokenize(paragraph)
    
    fts_list = x.paragraph_fts_as_txt_list
    
    fts_spans = []        
                
    for fts in fts_list:    
        
        fts_str_idx = fts.index("Structural Features:")
        fts = fts[fts_str_idx:]

        fts_tokenized = tokenizer.tokenize(fts)
        
        for i in range(len(paragraph_tokenized)):
            
            span = (i, i+len(fts_tokenized)-1)
            
            if paragraph_tokenized[span[0]:span[1]+1] == fts_tokenized and span not in fts_spans:
                
                fts_spans.append(span)    
                 
    return fts_spans

In [137]:
df['feature_spans_new'] = df.apply(lambda x: get_fts_spans_new(x), axis=1)

In [138]:
df['feature_spans_new']

0                                             [(98, 109)]
1            [(25, 36), (67, 78), (124, 135), (160, 171)]
2       [(21, 32), (48, 59), (86, 97), (175, 186), (21...
3                                              [(39, 50)]
4                                              [(77, 88)]
                              ...                        
1714                                           [(35, 46)]
1715                                           [(44, 55)]
1716    [(14, 25), (43, 54), (105, 116), (135, 146), (...
1717    [(14, 25), (55, 66), (94, 105), (115, 126), (1...
1718                                           [(18, 29)]
Name: feature_spans_new, Length: 1719, dtype: object

In [139]:
def sanity_check(x):
    
    return len(x.feature_spans_new) == len(x.paragraph_components_list) 

In [140]:
df['sanity_new'] = df.apply(lambda x: sanity_check(x), axis=1)

In [141]:
df['sanity_new'].value_counts()

True     1718
False       1
Name: sanity_new, dtype: int64

In [142]:
df.feature_spans_new[786]

[(20, 31)]

In [143]:
df.paragraph_components_list[786]

[['human needs for farmland, housing and industry are more important'],
 ['human needs for farmland, housing and industry are more important']]

In [144]:
df.paragraph_w_fts_as_txt[786]

'Nowadays, there is a prevailing opinion that human needs for farmland, housing and industry are more important [SEP] Structural Features: 1, Yes, Yes, Yes, No [SEP] than saving land for endangered animals. People who disagree with the point dispute that the decreasing of land for endangered animals will bring damage to ecological balance. As far as I am concerned, I agree with the opinion human needs for farmland, housing and industry are more important [SEP] 1, No, Yes, Yes, No [SEP]. The reasons are based on the following aspects.'

In [145]:
df.ac_spans_new[786], df.am_spans_new[786]

([(8, 18), (74, 84)], [(-1, -1), (-1, -1)])

#### dataset

In [146]:
train_df = df[df.split=='TRAIN'].reset_index(drop=True)
test_df = df[df.split=='TEST'].reset_index(drop=True)

In [147]:
dataset_train = Dataset.from_pandas(train_df)
dataset_test = Dataset.from_pandas(test_df)

In [148]:
train_val_datasets = dataset_train.train_test_split(train_size=0.8)
dataset_train = train_val_datasets['train']
dataset_val = train_val_datasets['test']

Parameter 'generator'=Generator(PCG64) of the transform datasets.arrow_dataset.Dataset.train_test_split couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [149]:
dataset = DatasetDict({"train": dataset_train, "test": dataset_test, "validation": dataset_val})

In [150]:
dataset

DatasetDict({
    train: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_w_fts_as_txt', 'ac_spans_new', 'sanity_new', 'am_spans_new', 'feature_spans_new'],
        num_rows: 1088
    })
    test: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_spans', 'paragraph_w_fts_as_txt', 'ac_spans_new', 'sanity_new', 'am_spans_new', 'feature_spans_new'],
        num_rows: 358
    })
    validation: Dataset({
        features: ['paragraph', 'paragraph_components_list', 'paragraph_labels_list', 'paragraph_markers_list', 'split', 'essay_nr', 'paragraph_fts_as_txt_list', 'paragraph_labels', 'paragraph_ac_spans', 'paragraph_am_sp

In [151]:
torch.save(dataset, os.path.join("/notebooks/KURI-BERT/notebooks/full_formula_w_fts", 'pe_dataset_for_bert_minus_w_fts_combined.pt'))